#### Combining Attack Angle and Zone

In [30]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data')
df = pd.read_csv('pitch/pitch_cleaned.csv').drop(columns=['Unnamed: 0'])

#### Ideal

In [32]:
print(df['attack_angle'].describe())
print(df['attack_angle'].quantile(0.03))
print(df['attack_angle'].quantile(0.97))

count    677102.000000
mean          9.150100
std          11.791634
min         -89.757838
25%           1.980554
50%           9.449831
75%          16.919260
max          87.258474
Name: attack_angle, dtype: float64
-12.627364848272745
29.56806044321247


In [33]:
bin_edges = [-np.inf] + list(np.arange(-14, 29.5, 1.5)) + [np.inf]

In [34]:
df['attack_bin'] = pd.cut(df['attack_angle'], bins=bin_edges,right=True )
woba = df.groupby(['zone', 'attack_bin'])['woba_value'].mean().reset_index()
woba_count = df.groupby(['zone', 'attack_bin']).size().reset_index()
woba = woba.merge(woba_count, on = ['zone', 'attack_bin'])
df['attack_bin'] =  df.groupby(['attack_bin', 'zone']).ngroup()

C:\Users\dalto\AppData\Local\Temp\ipykernel_28172\229022322.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  woba = df.groupby(['zone', 'attack_bin'])['woba_value'].mean().reset_index()
C:\Users\dalto\AppData\Local\Temp\ipykernel_28172\229022322.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  woba_count = df.groupby(['zone', 'attack_bin']).size().reset_index()
C:\Users\dalto\AppData\Local\Temp\ipykernel_28172\229022322.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=Tru

In [35]:
df.to_csv('pitch/pitch_encoded.csv', index=False)